# Spotify

We can get a lot of subtle stats on an automated bases daily using the Spotipy library. This notebook evaluates what metrics we might want to pull on a periodic basis. 

## Setup

### Imports

In [ ]:
import spotipy
import configparser
import pandas as pd
import numpy as np
import plotly.express as px
from spotipy.oauth2 import SpotifyClientCredentials as SCC

In [ ]:
country_codes = pd.read_csv('./data/country_codes.csv')
country_codes.head()

### Create Spotify object

In [ ]:
config = configparser.ConfigParser()
config.read('../../src/config.ini')

creds = SCC(client_id=config['SPOTIFY']['client_id'],
            client_secret=config['SPOTIFY']['client_secret'])
spotify = spotipy.Spotify(client_credentials_manager=creds)

## Querying Kontras

In [ ]:
kontras_uri = config['SPOTIFY']['kontras_uri']
fokof_uri = 'spotify:artist:7JL8PgOQe05PjPZtwBfR8x'
artist_uri = kontras_uri

In [ ]:
albums = spotify.artist_albums(artist_uri)
artist = spotify.artist(artist_uri)

In [ ]:
albums

In [ ]:
keys_to_keep = ['artist','available_markets',
                'disc_number','duration_ms',
                'explicit','name','track_number',
                'type','uri']

In [ ]:
import requests
import json

In [ ]:
def get_albums_count(album_ids):
    """
    Function that uses a server hosted on https://t4ils.dev:4433/api/beta/albumPlayCount 
    for https://github.com/evilarceus/Spotify-PlayCount that returns the playcount of an album
    
    Parameters
    album_ids(list): list of album ids, ['5NJafKlvdUSssGlVG3nGuo', '2w1W2OrF3PW4eAuqdN0nVk']
    """
    df = pd.DataFrame()
    for album_id in album_ids:
        resp = requests.get(f'https://t4ils.dev:4433/api/beta/albumPlayCount?albumid={album_id}')
        data = json.loads(resp.text)['data']
        df = pd.concat([df, pd.DataFrame(data)])
    return df

In [ ]:
df = get_albums_count(['5NJafKlvdUSssGlVG3nGuo', '2w1W2OrF3PW4eAuqdN0nVk'])

In [ ]:
df = df[['name', 'playcount']]

In [ ]:
for row in df.iterrows():
    print(row[1]['name'], row[1]['playcount'])

In [ ]:
albumid='5NJafKlvdUSssGlVG3nGuo'
resp = requests.get(f'https://t4ils.dev:4433/api/beta/albumPlayCount?albumid={albumid}')
data = json.loads(resp.text)['data']

In [ ]:
pd.DataFrame(data)

In [ ]:
[d['id'] for d in albums['items']]

In [ ]:
tracks_df = pd.DataFrame()
for album in albums['items']:
    print(f'Processing {album["name"]}')
    print(f'Plays: {get_album_playcount(album["id"])}')
    album_tracks = spotify.album_tracks(album['uri'])['items']
    for track in album_tracks:
        track['artist'] = track['artists'][0]['name']
        track['available_markets'] = ','.join(track['available_markets'])
        track_df = pd.DataFrame({k:track[k] for k in keys_to_keep}, index=[1])
        tracks_df = pd.concat([tracks_df, track_df])

In [ ]:
tracks_df

In [ ]:
tracks = spotify.tracks(tracks_df.uri.values) 

In [ ]:
pd.DataFrame(tracks['tracks'])

In [ ]:
spotify.

### Song Features

In [ ]:
audio_features = spotify.audio_features(tracks_df.uri.values[0:10])
audio_features_df = pd.DataFrame(audio_features).drop(['type', 'duration_ms'], axis=1)
df = tracks_df.merge(audio_features_df, on='uri')
df.loudness = np.log((df.loudness * -1)-2)
melted_features_df = df[['name','danceability','energy','loudness',
                      'speechiness','acousticness','instrumentalness',
                      'liveness','valence']].melt(id_vars='name')

In [ ]:
px.bar(melted_features_df,
       x='variable',
       y='value',
       color='name',
       barmode='group',
       title='Fokof Discography Spotify Attributes',
      template='plotly_dark')

## Playlists

In [ ]:
spotify.categories()

In [ ]:
arnold_uri = 'spotify:user:ueas3f07becuptaz4grd5huho'
andrew_uri = 'spotify:user:21r3nvcdxhyd4t5nzo3br56fy'
louwrens_uri = 'spotify:user:m7klrf290iwgyn4tm0qxlzfn0'
louwrens_id = 'm7klrf290iwgyn4tm0qxlzfn0'
fairly_local_playlist_uri = 'spotify:playlist:15vjNBeFwKvWcA1KyqStjU'

In [ ]:
playlists = spotify.user_playlists('Louwrens')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [ ]:
spotify.user_playlist(louwrens_id)

In [ ]:
spotify.user(andrew_uri)

In [ ]:
spotify.user_playlist_tracks(arnold_uri, fairly_local_playlist_uri)

In [ ]:
spotify.user_playlist_tracks(user=arnold_uri, playlist_id=)

In [ ]:
spotify.category_playlists('edm_dance')